In [36]:
import cv2
import json
import os
import numpy as np
import sys
import os

# Find the ERO-SNN folder and add it to the python path
current_dir = os.getcwd()

while os.path.basename(current_dir) != 'ERO-SNN':
    #print(os.path.basename(current_dir))
    current_dir = os.path.dirname(current_dir)
    
#print(f"Found ERO-SNN folder: {current_dir}")
sys.path.append(current_dir)

import BrianHF
from datasets.utils.parsing import batchIterator
from datasets.utils.events_representation import EROS
from datasets.utils.export import ensure_location, str2bool
from bimvee.importAe import importAe


In [37]:
def create_ts_list(fps,ts):
    out = dict()
    out['ts'] = list()
    x = np.arange(ts[0],ts[-1],1.0/fps)
    for i in x:
        out['ts'].append(i)
    return out

In [38]:
def process(data_dvs_file, output_path, skip=None, args=None):

    data_dvs = importAe(filePathOrName=data_dvs_file)
    print('File imported.')

    data_dvs = next(BrianHF.find_keys(data_dvs, 'dvs'))
    data_ts = create_ts_list(args['fps'],data_dvs['ts'])
    
    iterator = batchIterator(data_dvs, data_ts)
    
    frame_width = np.max(data_dvs['x'])+1
    frame_height = np.max(data_dvs['y'])+1
    eros = EROS(kernel_size=args['eros_kernel'], frame_width=frame_width, frame_height=frame_height)


    for fi, (events, _, batch_size) in enumerate(iterator):
        sys.stdout.write(f'frame: {fi}/{len(data_ts["ts"])}\r')
        sys.stdout.flush()
        
        # print(batch_size)
        for ei in range(batch_size):
            eros.update(vx=int(events['x'][ei]), vy=int(events['y'][ei]))

        frame = eros.get_frame()


        images_path =  os.path.join(output_path,'Images')
        ensure_location(images_path)
        path = os.path.join(images_path, f'frame_{fi:05d}.png')
        try:
            cv2.imwrite(path, frame)
        except:
            print('Error saving image')
            print(path)
            print(frame.shape)
            print(frame)
            return False
    return True

In [39]:
# Define the variables directly
eros_kernel = 4
skip_image = None
input_data_dir = 'InputData'
output_base_path = 'EROS'
write_images = True
frame_length = 1.1 #ms
interval_length = 30 #ms
fps = 33.0
dev = False
ts_scaler = 1.0

# Ensure the base output path exists
output_base_path = os.path.join(current_dir, output_base_path)
input_data_dir = os.path.join(current_dir, input_data_dir)

print('Input data directory: ', input_data_dir)
print('Output base path: ', output_base_path)

datasets = ['h36m_sample', 'MVSEC_short_outdoor'] #'EyeTracking']

# Create a dictionary to hold the arguments
args = {
    'eros_kernel': eros_kernel,
    'write_images': write_images,
    'frame_length': frame_length,
    'interval_length': interval_length,
    'fps': fps,
    'dev': dev,
    'ts_scaler': ts_scaler
    }

log_files = ['h36m_sample/cam2_S1_Directions/ch0dvs/data.log',
         'MVSEC_short_outdoor/leftdvs/data.log'] # 'EyeTracking/user_5_1/ch0dvs_old/data.log']

Input data directory:  /home/bernard/Desktop/IIT_Thesis/ERO-SNN/InputData
Output base path:  /home/bernard/Desktop/IIT_Thesis/ERO-SNN/EROS


In [40]:
for i, log_file in enumerate(log_files):
    # Create a corresponding output path
    log_file = os.path.join(input_data_dir, log_file)
    output_path = os.path.join(output_base_path, datasets[i])
    print("Processing:", log_file)
    print("Output path:", output_path)
    
    if process(log_file, output_path, skip=skip_image, args=args):
        print(f"Processed {log_file}")
    else:
        print(f"Error processing {log_file}")

Processing: /home/bernard/Desktop/IIT_Thesis/ERO-SNN/InputData/h36m_sample/cam2_S1_Directions/ch0dvs/data.log
Output path: /home/bernard/Desktop/IIT_Thesis/ERO-SNN/EROS/h36m_sample
{'filePathOrName': '/home/bernard/Desktop/IIT_Thesis/ERO-SNN/InputData/h36m_sample/cam2_S1_Directions/ch0dvs/data.log'}
{'filePathOrName': '/home/bernard/Desktop/IIT_Thesis/ERO-SNN/InputData/h36m_sample/cam2_S1_Directions/ch0dvs/data.log'}
importIitYarp trying path: /home/bernard/Desktop/IIT_Thesis/ERO-SNN/InputData/h36m_sample/cam2_S1_Directions/ch0dvs


  1%|          | 302/32375 [00:00<00:10, 3010.47it/s]

100%|██████████| 32375/32375 [00:05<00:00, 5606.95it/s]


Examining info.log: /home/bernard/Desktop/IIT_Thesis/ERO-SNN/InputData/h36m_sample/cam2_S1_Directions/ch0dvs/info.log
File imported.
Processed /home/bernard/Desktop/IIT_Thesis/ERO-SNN/InputData/h36m_sample/cam2_S1_Directions/ch0dvs/data.log
Processing: /home/bernard/Desktop/IIT_Thesis/ERO-SNN/InputData/MVSEC_short_outdoor/leftdvs/data.log
Output path: /home/bernard/Desktop/IIT_Thesis/ERO-SNN/EROS/MVSEC_short_outdoor
{'filePathOrName': '/home/bernard/Desktop/IIT_Thesis/ERO-SNN/InputData/MVSEC_short_outdoor/leftdvs/data.log'}
{'filePathOrName': '/home/bernard/Desktop/IIT_Thesis/ERO-SNN/InputData/MVSEC_short_outdoor/leftdvs/data.log'}
importIitYarp trying path: /home/bernard/Desktop/IIT_Thesis/ERO-SNN/InputData/MVSEC_short_outdoor/leftdvs


100%|██████████| 38292/38292 [00:07<00:00, 5409.92it/s] 


Examining info.log: /home/bernard/Desktop/IIT_Thesis/ERO-SNN/InputData/MVSEC_short_outdoor/leftdvs/info.log
File imported.
Processed /home/bernard/Desktop/IIT_Thesis/ERO-SNN/InputData/MVSEC_short_outdoor/leftdvs/data.log
